In [2]:
import json
import os
import sys

cur_path = os.path.abspath('__file__')
cur_dir  = os.path.dirname(cur_path)
par_dir  = os.path.dirname(cur_dir)
gra_dir  = os.path.dirname(par_dir)
sys.path.append(cur_dir)
sys.path.append(par_dir)
sys.path.append(gra_dir)

print("cur_dir = ", cur_dir)
print("par_dir = ", par_dir)
print("gra_dir = ", gra_dir)

cur_dir =  C:\Users\90311\Documents\code\MWP\math_graph2tree\math23k\src
par_dir =  C:\Users\90311\Documents\code\MWP\math_graph2tree\math23k
gra_dir =  C:\Users\90311\Documents\code\MWP\math_graph2tree


In [3]:
data_dir  = os.path.join(par_dir, "data")
data_path = os.path.join(data_dir, "test23k_processed_utf8.json")
print("data_path = ", data_path)

data_path =  C:\Users\90311\Documents\code\MWP\math_graph2tree\math23k\data\test23k_processed_utf8.json


In [4]:
with open(data_path, mode="r", encoding="utf-8") as reader:
    input_data = json.load(reader)
    
import prettytable as pt
item0 = input_data[0]
tb = pt.PrettyTable()
tb.field_names = ["name", "content"]
## 按行添加数据
for key, value in item0.items():
#     print("{}: {}".format(key, value))
#     print("%s : %s" % (key, value))
    tb.add_row([key, value])
print(tb)

+---------------------------+------------------------------------------------------------------------------------------------------------------------------+
|            name           |                                                           content                                                            |
+---------------------------+------------------------------------------------------------------------------------------------------------------------------+
|       original_text       |                    在一正方形花池的4周栽了44棵柳树，每两棵柳树之间的间隔是20米，这个正方形的周长=多少米？                    |
|          num_list         |                                                         [44.0, 20.0]                                                         |
|            text           | 在 一 正方形 花池 的 四周 栽 了 temp_a 棵 柳树 ， 每 两棵 柳树 之间 的 间隔 是 temp_b 米 ， 这个 正方形 的 周长 = 多少 米 ？ |
|         new_split         |     在 一 正方形 花池 的 四周 栽 了 44 棵 柳树 ， 每 两棵 柳树 之间 的 间隔 是 20 米 ， 这个 正方形 的 周长 = 多少 米 ？     |
|          equation    

In [5]:
def seg_and_tag(st, nums, nums_fraction):  # seg the equation and tag the num
    res = []
    for n in nums_fraction:
        if n in st:
            p_start = st.find(n)
            p_end   = p_start + len(n)
            if p_start > 0:
                res += seg_and_tag(st[:p_start], nums, nums_fraction)
            if nums.count(n) == 1:
                res.append("N"+str(nums.index(n)))
            else:
                res.append(n)
            if p_end < len(st):
                res += seg_and_tag(st[p_end:], nums, nums_fraction)
            return res

    pos_st = re.search("\d+\.\d+%?|\d+%?", st)
    if pos_st:
        p_start = pos_st.start()
        p_end = pos_st.end()
        if p_start > 0:
            res += seg_and_tag(st[:p_start], nums, nums_fraction)
        st_num = st[p_start:p_end]
        if nums.count(st_num) == 1:
            res.append("N"+str(nums.index(st_num)))
        else:
            res.append(st_num)
        if p_end < len(st):
            res += seg_and_tag(st[p_end:], nums, nums_fraction)
        return res
    for ss in st:
        res.append(ss)
    return res

In [6]:
import re
def transfer_num(data):
    print("test transfer_num function")
    print("Transfer numbers...")
    print("\n")
    pattern = re.compile("\d*\(\d+/\d+\)\d*|\d+\.\d+%?|\d+%?")  # 匹配文本中的所有数字

    pairs = []
    generate_nums = []
    generate_nums_dict = {}
    copy_nums = 0
    for d in data:
        nums = []
        input_seq = []

        seg       = d["segmented_text"].strip().split(" ")
        equations = d["equation"][2:]  # remove "x="
        
        print("debug segmented_text")
        print("segmented_text = ", d["segmented_text"])
        print("seg = ", seg)
        print("\n")
        
        print("debug equation")
        print("equation = ", d["equation"])
        print("equations = ", equations)
        print("\n")
        
        for s in seg:
            pos = re.search(pattern, s)
            if pos and pos.start() == 0:
                nums.append(s[pos.start(): pos.end()])
                input_seq.append("NUM")
                if pos.end() < len(s):
                    input_seq.append(s[pos.end():])
            else:
                input_seq.append(s)
        
        print("debug input_seq")
        # 文本中的所有数字都被替换为NUM标识
        print("input_seq = ", input_seq)
        print("\n")
        
        print("debug nums")
        print("nums = ", nums)  # 文本中出现的数字
        print("\n")
        
        if copy_nums < len(nums):
            copy_nums = len(nums)  # 文本中出现的数字个数

        nums_fraction = []
        for num in nums:
            if re.search("\d*\(\d+/\d+\)\d*", num):
                nums_fraction.append(num)
        nums_fraction = sorted(nums_fraction, key=lambda x: len(x), reverse=True)

        print("debug nums_fraction")
        print("nums_fraction = ", nums_fraction)  # 文本中出现的分数
        print("\n")
        
        out_seq = seg_and_tag(st=equations, nums=nums, nums_fraction=nums_fraction)
        for s in out_seq:  # tag the num which is generated
            if s[0].isdigit() and s not in generate_nums and s not in nums:
                generate_nums.append(s)
                generate_nums_dict[s] = 0
            if s in generate_nums and s not in nums:
                generate_nums_dict[s] = generate_nums_dict[s] + 1

        print("debug generate_nums_dict")
        print("out_seq = ", out_seq)
        print("generate_nums_dict = ", generate_nums_dict)  # 常数字典
        print("\n")
        
        num_pos = []
        for i, j in enumerate(input_seq):
            if j == "NUM":
                num_pos.append(i)
        assert len(nums) == len(num_pos)
        
        print("debug num_pos")
        print("num_pos = ", num_pos)
        print("\n")
        
        pairs.append((input_seq, out_seq, nums, num_pos))
    
    temp_g = []
    for g in generate_nums:
        if generate_nums_dict[g] >= 5:
            temp_g.append(g)
    print("debug temp_g")
    print("temp_g = ", temp_g)
    print("copy_nums = ", copy_nums)
    print("\n")
    return pairs, temp_g, copy_nums

test_data = input_data[:1]
pairs, generate_nums, copy_nums = transfer_num(test_data)

test transfer_num function
Transfer numbers...


debug segmented_text
segmented_text =  在 一 正方形 花池 的 4 周 栽 了 44 棵 柳树 ， 每 两棵 柳树 之间 的 间隔 是 20 米 ， 这个 正方形 的 周长 = 多少 米 ？
seg =  ['在', '一', '正方形', '花池', '的', '4', '周', '栽', '了', '44', '棵', '柳树', '，', '每', '两棵', '柳树', '之间', '的', '间隔', '是', '20', '米', '，', '这个', '正方形', '的', '周长', '=', '多少', '米', '？']


debug equation
equation =  x=44*20
equations =  44*20


debug input_seq
input_seq =  ['在', '一', '正方形', '花池', '的', 'NUM', '周', '栽', '了', 'NUM', '棵', '柳树', '，', '每', '两棵', '柳树', '之间', '的', '间隔', '是', 'NUM', '米', '，', '这个', '正方形', '的', '周长', '=', '多少', '米', '？']


debug nums
nums =  ['4', '44', '20']


debug nums_fraction
nums_fraction =  []


debug generate_nums_dict
out_seq =  ['N1', '*', 'N2']
generate_nums_dict =  {}


debug num_pos
num_pos =  [5, 9, 20]


debug temp_g
temp_g =  []
copy_nums =  3




In [7]:
for p in pairs:
    print("p[0] = (input_seq)")
    print("p[0] = ", p[0])
    print("p[1] = (output_seq)")
    print("p[1] = ", p[1])
    print("p[2] = (numbers in text)")
    print("p[2] = ", p[2])
    print("p[3] = (number_pos in text)")
    print("p[3] = ", p[3])
    break

p[0] = (input_seq)
p[0] =  ['在', '一', '正方形', '花池', '的', 'NUM', '周', '栽', '了', 'NUM', '棵', '柳树', '，', '每', '两棵', '柳树', '之间', '的', '间隔', '是', 'NUM', '米', '，', '这个', '正方形', '的', '周长', '=', '多少', '米', '？']
p[1] = (output_seq)
p[1] =  ['N1', '*', 'N2']
p[2] = (numbers in text)
p[2] =  ['4', '44', '20']
p[3] = (number_pos in text)
p[3] =  [5, 9, 20]
